## Input Data Format
Marvel-Graph.txt: \
_A hero may span multiple lines!_

    4395 2237 1767 472 4997 5931 6235 1478 1369 806 3994 6232 
    3519 4704 2460 763 1602 5306 5358 6121 6160 2459 3173 4963 6166 
    3518 5409 
Marvel-Names.txt:
    
    5300 "SPENCER, TRACY"
    5301 "SPERZEL, ANTON"
    5302 "SPETSBURO, GEN. YURI"
    5303 "SPHINX"
    5304 "SPHINX II"
    5305 "SPHINX III"
    5306 "SPIDER-MAN/PETER PAR"
    5307 "SPIDER-MAN III/MARTH"
    5308 "SPIDER-MAN CLONE/BEN"
    5309 "SPIDER-WOMAN/JESSICA"
    
## Most popular superhero: Strategy
- Map input data to (heroID, number of co-occurences) per line
- add up  co-occurence by heroID using reduceByKey()
- flip (map) RDD to (number, heroID) so we can..
- Use max() on the RDD to find the hero with the most co-occurences
- Look up the name of the winner and display the result

In [1]:
from pyspark import SparkContext

sc = SparkContext("local", "PopularHero")

def countCoOccurences(line):
    elements = line.split()
    return (int(elements[0]), len(elements) - 1)

def parseNames(line):
    fields = line.split('\"')
    return (int(fields[0]), fields[1].encode("utf8"))

In [4]:
names = sc.textFile("../data/Marvel-Names.txt")
namesRdd = names.map(parseNames)

lines = sc.textFile("../data/Marvel-Graph.txt")

pairings = lines.map(countCoOccurences)
totalFriendsByCharacter = pairings.reduceByKey(lambda x, y : x + y)
flipped = totalFriendsByCharacter.map(lambda xy : (xy[1], xy[0]))

mostPopular = flipped.max()

mostPopularName = namesRdd.lookup(mostPopular[1])[0]

print(str(mostPopularName) + " is the most popular superhero, with " + \
    str(mostPopular[0]) + " co-appearances.")


b'CAPTAIN AMERICA' is the most popular superhero, with 1933 co-appearances.
